# Group 2 - Dorukhan Kılınç - Ahmet Buğra Taksuk

# Libraries and Input-Output Functions

All the packages needed are imported below. To install them open the terminal and run pip install package_name

In the directory of this notebook, we need to have the folders Inputs and Outputs, otherwise the input readings and output writings would not work.

In [37]:
# to list the files in a directory to read inputs
from os import listdir

# for dataframes
import pandas as pd

#for array operations
import numpy as np

#for random number generation
import random

#to measure time
import time

#for poisson distribution
from scipy.stats import poisson

#for progress bars in for loops
from tqdm import tqdm

In [3]:
#A function to write the degree sequences created to txt files with the desired format
def write_input(input_index,method, sequence):
    seq = list(sequence)
    file_name = 'Inputs/2-{}-{}-{}.txt '.format(len(seq),input_index, method)
    with open(file_name, 'w') as f:
        f.write(f'{len(seq)}\n')
        f.write('{}'.format(seq).replace('[', '').replace(']','').replace(',',''))
    f.close()
    return

In [73]:
#writes the neighborhood list of a given realization to a txt file in the required format.
def write_output(input_index, sequence, neigh_list, method):
    seq = list(sequence)
    file_name = 'Outputs/O-2-{}-{}-{}.txt '.format(len(seq), input_index, method)
    with open(file_name, 'w') as f:
        #for computational reasons, pairing model is limited to 5 minutes of runtime(see the code of it), it returns 'Failed' in these cases. 
        if neigh_list == 'Failed': f.write('Failed')
        else:
            f.write(f'{len(seq)}\n')
            f.write('{}\n'.format(seq).replace('[', '').replace(']','').replace(',',''))
            vertex = 1
            for i in neigh_list : 
                #np arrays start with index 0 whereas we are asked to label vertices starting with 1, hence the +1
                f.write(f'{vertex} '+'{}\n'.format(list(np.array(i)+1)).replace('[', '').replace(']','').replace(',',''))
                vertex = vertex + 1
    f.close()
    return


#writes 0 for not graphical sequences 
def write_0(input_index, sequence, method):
    seq = list(sequence)
    file_name = 'Outputs/O-2-{}-{}-{}.txt '.format(len(seq), input_index, method)
    with open(file_name, 'w') as f:
        f.write('0')
    f.close()
    return

# Some Basic Functions and Recursive Implementations of Havel-Hakimi Theorem

## Implementation of Havel-Hakimi Theorem to Check if a Degree Sequence is Graphical

To implement the Havel-Hakimi theorem to check if a given degree sequence is graphical, we start by checking some basic conditions.
- Firstly, if the sequence consist of negative integers, it is obviously not graphical.
- Secondly, a degree sequence is not graphical if the sum of degrees in this sequence is an odd number. This is simply the violation of the handshaking lemma.
- Lastly, for each entry in a given degree sequence, there must be at least that much positive entries other than the degree given by this entry because each degree d requires d many vertices.

We now proceed to implementation of the algorithm. The above 3 conditions will be referred as basic conditions.

### Algorithm
- Input: A degree sequence
- Output: True if the degree sequence is graphical, False otherwise
- Algorithm:
1) If the sequence is the 0 vector, return True, terminate the algorithm.
2) If the sequence doesnt provide the basic conditions, return False, terminate the algorithm.
3) Sort this sequence, call it s'.
4) Delete first entry of s', let d be the degree corresponding to the degree of this deleted element, subtract 1 from the first d entries. Call this sequence s''
5) Call the algorithm by providing s''.

In [5]:
#returns whether or not the sequence has any negative entires
def count_negative_degrees(sequence):
    return (sum(sequence < 0))

#returns whether or not the sum of degrees is even
def is_total_degree_even(sequence):
    return (sum(sequence)%2==0)

#combines above 2 and the condition that for every vertex with positive degree there must at least be that much vertices having positive degrees.
#It is enough to check this condition for the vertex with maximum degree.
def basic_check(sequence):
    return is_total_degree_even(sequence) & (count_negative_degrees(sequence) == 0) & (max(sequence) < np.count_nonzero(sequence))

#this function returns whether or not a degree sequence is graphical or not using havel-hakimi theorem.
def is_graphical(sequence):
    
    #if the degree sequence only consists of zeros, we are done, the sequence is graphic.
    if np.sum(sequence) == 0 : return True
    
    elif (basic_check(sequence)== False): return False
    
    #sort the degree sequence, note: numpy sorts arrays in ascending order so - signs added for descending order.
    sorted_seq = -np.sort(-sequence)
    
    new_seq = sorted_seq.copy()
        
    #note the degree of the vertex that will be deleted, say d, to subtract 1 from d many vertices.
    edges_to_delete = new_seq[0]
        
    #delete d edges from d vertices of highest degree, start from the second vertex of highest degree
    for i in range(1, edges_to_delete+1):
        new_seq[i] = new_seq[i] - 1 
        
    #call the function with the new sequence without the first vertex
    return is_graphical(new_seq[1:])
    

In [6]:
a = np.array([1,2,3,2])

is_graphical(a)

True

## Implementation of Havel-Hakimi Theorem to Find a Realization of a Given Degree Sequence

This section provides an algorithm to find a deterministic realization of a graphical degree sequence using Havel-Hakimi theorem. 

Only addition in this function is that we start with an empty edge set and whenever we delete a vertex v and subtract 1 from the degree of a vertex w, we add (v,w) to the edge set.

In [7]:
def havel_hakimi_realization(sequence, edge_list):
    
    #if the degree sequence only consists of zeros, we are done.
    if np.sum(sequence) == 0 : return edge_list
    
    #sort the degree sequence, note: numpy sorts arrays in ascending order so - signs added for descending order.
    sorted_seq = -np.sort(-sequence)
    #apply this sorting to the vertex labels, i.e. indices of the sequence as well.
    sorted_indices = (-sequence).argsort()
    
    #note the degree of the vertex that will be deleted, say d, to subtract 1 from d many vertices.
    edges_to_delete = sorted_seq[0]
    vertex_to_delete = sorted_indices[0]
    
    new_seq = sequence.copy()
    new_seq[vertex_to_delete] = 0
    
    #delete d edges from d vertices of highest degree, start from the second vertex of highest degree
    #here the variable vertex represents the position of the vertex with ith highest degree in the original degree sequence.
    for i in range(1, edges_to_delete+1):
        vertex = sorted_indices[i]
        new_seq[vertex] = new_seq[vertex] - 1
        edge_list.append([vertex_to_delete, vertex])
    
    #call the function with the new sequence
    return havel_hakimi_realization(new_seq, edge_list)
    

In [8]:
a = np.array([2,3,4,3,4])

havel_hakimi_realization(a,[])

[[2, 4], [2, 1], [2, 3], [2, 0], [4, 1], [4, 3], [4, 0], [1, 3]]

# Pairing Model

Although the steps of the pairing model is pretty similar to the that of explained in the article, we added a few control mechanisms to reduce the computation time. Before that, let us define some terms to refer further in the explanation:

**Vertex Array:** The array of vertices gotten by taking each vertex with label i deg(i) times. Here, the labels are the indices of the entries in the original degree sequence.

**Remaining Degree Sequence:** The degree sequence we get by adding the edge (v,w), i.e. subtracting 1 from the entries v and w in the degree sequence.

**Adjacency List:** A list consisting of lists denoting the neighborhood for each vertex. For example, 2nd entry of the adjacency list is a list denoting the neighborhood of the 2nd vertex.

## Added Control Mechanisms: 

- Firstly, we only consider the pairings which dont correspond to a loop or an existing edge by taking a vertex v from the vertex array and then limiting the second choice to the part of the vertex array which dont correspond to any vertex in the neighborhood of v.
- Secondly, we add the edge if the remaning degree sequence is still graphical.(using havel-hakimi theorem)
- Lastly, we restart the algorithm whenever in the remaining degree sequence and the adjaceny list we are guarenteed to end up with a loop or an edge which is already formed.


## Algorithm:

- Input: A graphical degree sequence

- Output: Adjacency list

- Steps:
    1) Create a vertex array from the input degree sequence, a degree sequence equal to the input degree sequence and an empty adjacency list.
    2) If the desired number of edges is acquired, return adjacency list and terminate the algorithm.
    3) If the vertex array and adjacency list definitely correspond to a multigraph, go to step 1.
    4) Add a pairing in the vertex array to the graph if the edge implied by this pairing doesnt correspond to a loop or an existing edge and keeps the remaining degree sequence graphical. Update the adjacency list, and delete the vertices of this edge from the vertex array. Update the degree sequence by subtracting one to the corresponding edges. Go to step 2.

In [35]:
#subtract 1 from ith and jth entries in a given sequence
def subtract_one(i, j, sequence):
    seq = sequence.copy()
    seq[i] = seq[i] - 1
    seq[j] = seq[j] - 1
    return seq


#create an array consisting each point v degree(v) times.
def create_vertex_array(sequence):
    vertex_array = []
    for vertex in range(len(sequence)):
        vertex_array = vertex_array + [vertex]*sequence[vertex]
    return np.array(vertex_array)   

#iterate over vertices, if there exists a vertice whose connection to another vertex definitely corresponds to a not simple graph, return true. 
def if_graph_is_definitely_multigraph(vertex_list, adjacency_list):
    
    for vertex in np.unique(vertex_list):
        
        #self loop and multi edges are not allowed
        not_allowed_pts = [vertex] + adjacency_list[vertex]
        #the rest is allowed vertices
        allowed_pts = vertex_list[np.isin(vertex_list,not_allowed_pts, invert = True)]
        
        if allowed_pts.size == 0: return True
    
    return False

#return a pair of valid vertices s.t.
#    they dont correspond to the same point(no loop)
#    they are not yet adjacent(no multigraphs)
#the first vertex is chosen to be with the largest degree to reduce number of reiterations(see the paper for more explanation)
def return_valid_pairs(vertex_list,sequence, adjacency_list):
    
        v1 = np.argmax(sequence)
        
        not_allowed_pts = [v1] + adjacency_list[v1]

        allowed_pts = vertex_list[np.isin(vertex_list,not_allowed_pts, invert = True)]
        
        v2 = np.random.choice(allowed_pts)
        return (v1,v2)

#update the vertex array, degree sequence and adjacency list according to a given valid pair if they correspond to a valid edge
#a valid edge is basically an edge whose addition would not disrupt the graphicallity of the remanining degree sequence
def add_valid_edge(vertex_array, sequence, adjacency_list):
    while(True):
        
        v1,v2 = return_valid_pairs(vertex_array,sequence, adjacency_list)
            
        #condition for not adding the edge (v1,v2):
        #    the addition of (v1,v2) makes the remaining degree sequence not graphical.  
        if (not is_graphical(subtract_one(v1, v2, sequence))): continue
        
        #else, add the edge, remove the vertices matched, update the degree sequence.
        else:
            #add v2 to the neighborhood of v1 and vice versa
            new_adjacency_list = adjacency_list.copy()
            new_adjacency_list[v1] = new_adjacency_list[v1] + [v2]
            new_adjacency_list[v2] = new_adjacency_list[v2] + [v1]

            #update the degree sequence
            new_seq = subtract_one(v1, v2, sequence)
            
            #delete v1 and v2 from the vertex list since they are paired. below codes delete the first instaces of v1 and v2 from the list.
            new_vertex_array = np.delete(vertex_array, np.where(vertex_array == v1)[0][0])
            new_vertex_array = np.delete(new_vertex_array, np.where(new_vertex_array == v2)[0][0])
            
            return (new_vertex_array, new_seq, new_adjacency_list)

def pairing_model(sequence, t):
    
    while(True):
        
        vertex_array = create_vertex_array(sequence)
        
        edges_added = 0
        
        #total number of edges to be created. note that each edge is counted twice when we sum the degrees(handshaking lemma)
        total_edges = sum(sequence)/2
        
        #create a copy of the original sequence. this new sequence will be updated when a new edge is added.
        new_seq = sequence.copy()
        
        adjacency_list = [[]]*len(sequence)
        while(True):
            if edges_added == total_edges: return adjacency_list
            elif time.time() - t > 300: return 'Failed'
            elif if_graph_is_definitely_multigraph(vertex_array, adjacency_list):
                break
            else:
                vertex_array, new_seq, adjacency_list = add_valid_edge(vertex_array, new_seq, adjacency_list)
                
                edges_added = edges_added + 1
        

# Sequential Algorithm

Call this algorithm with a graphical degree sequence and an empty edge set.

- Input: A graphical degree sequence, an edge set
- Output: An edge set
- Algorithm:
1) If the sequence is the empty sequence, return the edge set, terminate the algorithm.
2) Let v be the vertex implied by the minimum nonzero entry of the sequence. 
3) Create an empty candidate list.
4) Iterate over the sequence:
5)     If (w,v), w !=v is not in the edge list and the remaining degree sequence is graphical: add w to the candidate list
6) Assign probabilites to the candidate vertices proportional to their degree. 
7) Randomly chose a candidate according to the given candidate probabilities, call u.
8) Call the algorithm by providing a sequence in which the vertices v and w has 1 less degree and the edge set updated by adding the edge (u,w)

In [10]:
#subtract 1 from ith and jth entries in a given sequence
def subtract_one(i, j, sequence):
    seq = sequence.copy()
    seq[i] = seq[i] - 1
    seq[j] = seq[j] - 1
    return seq

#since the sequence will have 0 entries at each step, np.argmin would give us the index of the first 0 entry.
#therefore, we mask the 0 entries firstly to omit them in the argmin search.
def minimum_nonzero(sequence):
    sequence_zeros_masked = np.ma.array(sequence, mask =(sequence == 0))
    return sequence_zeros_masked.argmin()


#main part of the algorithm.
#call it with a sequence without specifying an edge_list, this argument is used in intermediary steps
def seq_alg(sequence, edge_list):
    
    new_seq = sequence.copy()
    start_time = time.time()
    while(True):
        #algorithm will stop when the degree sequence is a 0 array.
        if sum(new_seq)==0 or (time.time() - start_time > 60): return edge_list
        
        
        #min_d is the min degree vertex with nonzero degree
        min_d = minimum_nonzero(new_seq)

        #start an empty candidate list
        candidate_list = []
        
        for j in range(len(new_seq)):
            #candidate conditions:
            #    j and min_d are not the same vertex
            #    the vertex (min_d, j) is not already contained in the edge set
            #    the degree sequence obtained by subtracting 1 from these 2 vertices is also graphical
            if((min_d != j) &([min_d, j] not in edge_list) & ([j, min_d] not in edge_list) & is_graphical(subtract_one(min_d, j, new_seq))):
                candidate_list.append(j)
        
        #each candidate is assigned a probability proportional to its degree
        #prob_i = degree(i)/sum(degree(j), j in candidate list)
        candidate_prob_list = None
        candidate_prob_list = new_seq[candidate_list]/ sum(new_seq[candidate_list])
        
        
        #a vertex from the candidate list is chosen at random according to the probabilities given
        vertex_chosen = np.random.choice(candidate_list, p=candidate_prob_list)
        
        #edge list is updated
        edge_list.append([min_d, vertex_chosen])
        
        #a new degree sequence with one less degrees at the entries of the edge above is created
        new_seq = subtract_one(min_d, vertex_chosen, new_seq)
        

        

In [11]:
a = np.array([2,3,4,3,4])
seq_alg(a, [])

[[0, 2], [0, 4], [1, 3], [1, 2], [1, 4], [2, 4], [2, 3], [3, 4]]

# Neigborhood Lists

To get the neighborhood list from a given degree sequence and edge set.

In [12]:
def edge_set_to_neigborhood_list(sequence, edge_set):
    #start with an empty neighborhood list
    neighborhood_list = []
    
    #iterate over the vertices, note that the label of the vertex corresponds to the index of the degree sequence
    for vertex in range(len(sequence)):
        
        #create an empty neighborhood list for this vertex
        vertex_neighborhood = []
        
        #iterate over the edges
        for edge in edge_set:
            
            # vertex in edge implies that the other vertex in this edge is in the neighborhood of the vertex 
            if vertex in edge:
                
                #take the difference between sets created from edge and vertex, i.e. the other vertex, append it to the vertex neighborhood list.
                vertex_neighborhood.append(list(set(edge)-set([vertex]))[0])
        
        #append the neighborhood of this vertex to the neighborhood list. 
        #Note that by iterating over each vertex we will end up with a list whose ith entry is a list with vertices adjacent to vertex i.
        neighborhood_list.append(vertex_neighborhood)
        
    return neighborhood_list

In [13]:
a = np.array([2,3,4,3,4])
edge_set_to_neigborhood_list(a,seq_alg(a,[])) 

[[2, 4], [3, 2, 4], [0, 1, 4, 3], [1, 2, 4], [0, 1, 2, 3]]

# Random Degree Sequence Generation

In [14]:
#usual random number generation with a seed and upper bound(number of vertices) parameter
def seq_generator_uniform(num_vertices, seed):
    np.random.seed(seed)
    return np.random.randint(0,num_vertices, num_vertices)

In [15]:
#sequence generator using erdos-renyi method where probability of a vertex having degree k is proportional to a poisson var. with parameter lambda taking the value k
def seq_generator_erdos_renyi(num_vertices, mean_edge, seed):
    degrees = np.arange(num_vertices)
    degree_probs = []
    #degree_probs[i] corresponds to the probability that the vertex has degree i
    for degree in degrees:
        degree_probs.append(poisson.pmf(degree, mu = mean_edge))
    degree_probs = np.array(degree_probs)
    #normalize it so that the probabilities sum to 1.
    degree_probs = degree_probs/degree_probs.sum()
    np.random.seed(seed)
    return(np.random.choice(degrees, size = num_vertices, p = degree_probs))

In [16]:
#sequence generator using scale-free method where the probability of a vertex having degree k is proportional to k^(-gamma) where gamma is the parameter of this func.
def seq_generator_scalefree(num_vertices, gamma, seed):
    #create an array of possible degrees
    degrees = np.arange(1, num_vertices, dtype = float)
    
    #probability array is the array whose ith element is that of array above raised to the power -gamma
    degree_probs = np.power(degrees, -gamma)
    degrees = np.array(degrees, dtype = int)
    
    #normalize probabilities so that they sum to one
    degree_probs = degree_probs/degree_probs.sum()
    
    np.random.seed(seed)
    return(np.random.choice(degrees, size = num_vertices, p = degree_probs))

In [75]:
# this block is for performance comparison of random degree sequence generation methods
# we note the time elapsed during the generation of a degree sequence using each method, and whether or not it is graphical

methods = []
params = []
number_of_vertices = []
degrees = []
graphical = []
times = []

input_index = 1

for vertices in [100, 250, 500, 1000]:
    for seed in range(100):
        
        t = time.time()
        seq1 = seq_generator_uniform(vertices, seed)
        times.append(time.time()-t)
        
        degrees.append(sum(seq1)/(2*vertices))
        graphical.append(is_graphical(seq1))
        
        t = time.time()
        seq2 = seq_generator_scalefree(vertices, 3, seed)
        times.append(time.time()-t)
        
        degrees.append(sum(seq2)/(2*vertices))
        graphical.append(is_graphical(seq2))
        
        t = time.time()
        seq3 = seq_generator_erdos_renyi(vertices, 4, seed)
        times.append(time.time()-t)
        
        degrees.append(sum(seq3)/(2*vertices))
        graphical.append(is_graphical(seq3))
        
        t = time.time()
        seq4 = seq_generator_erdos_renyi(vertices, 5, seed)
        times.append(time.time()-t)
        
        degrees.append(sum(seq4)/(2*vertices))
        graphical.append(is_graphical(seq4))
        
        t = time.time()
        seq5 = seq_generator_erdos_renyi(vertices, 6, seed)
        times.append(time.time()-t)
        
        degrees.append(sum(seq5)/(2*vertices))
        graphical.append(is_graphical(seq5))
    
        methods = methods + ['uniform', 'scale-free','Erdos-Renyi',
                            'Erdos-Renyi','Erdos-Renyi']
        params = params + [0, 3, 4, 5, 6]
        number_of_vertices = number_of_vertices+[vertices]*5

In [76]:
# dataframe generation for aggregate statistics
d = {'Method': methods,
    'Parameter': params,
    'Number of Vertices': number_of_vertices,
    'Ratio of Graphical Sequences': graphical,
    'Average Degree': [2*degree for degree in degrees],
    'Average Time Elapsed During Sequence Generation': times}
df = pd.DataFrame(d)

In [77]:
# aggregate statistics
df.groupby(['Method', 'Parameter','Number of Vertices']).mean()

Ratio of Graphical Sequences  \
Method      Parameter Number of Vertices                                 
Erdos-Renyi 4         100                                         0.46   
                      250                                         0.51   
                      500                                         0.50   
                      1000                                        0.53   
            5         100                                         0.46   
                      250                                         0.48   
                      500                                         0.54   
                      1000                                        0.52   
            6         100                                         0.57   
                      250                                         0.40   
                      500                                         0.46   
                      1000                                        0.55   
scale-free  3         100                                         0.54   
                      250                                         0.39   
                      500                                         0.48   
                      1000                                        0.53   
uniform     0         100                                         0.11   
                      250                                         0.04   
                      500                                         0.03   
                      1000                                        0.04   

                                          Average Degree  \
Method      Parameter Number of Vertices                   
Erdos-Renyi 4         100                        3.97520   
                      250                        3.97564   
                      500                        3.97952   
                      1000                       3.98777   
            5         100                        4.97380   
                      250                        4.97088   
                      500                        4.97800   
                      1000                       4.98794   
            6         100                        5.96610   
                      250                        5.96608   
                      500                        5.97352   
                      1000                       5.98547   
scale-free  3         100                        1.34680   
                      250                        1.35804   
                      500                        1.35560   
                      1000                       1.36455   
uniform     0         100                       49.36090   
                      250                      124.82216   
                      500                      249.68264   
                      1000                     498.98622   

                                          Average Time Elapsed During Sequence Generation  
Method      Parameter Number of Vertices                                                   
Erdos-Renyi 4         100                                                        0.008628  
                      250                                                        0.020189  
                      500                                                        0.040804  
                      1000                                                       0.079847  
            5         100                                                        0.008417  
                      250                                                        0.020099  
                      500                                                        0.041301  
                      1000                                                       0.081422  
            6         100                                                        0.008399  
                      250                                                       

# Input Generation

The code below is for the generation of the input sequences. We generate 20 vertices for each of the following setting with 25, 50, 75 and 100 vertices:
- Using scale free method with $\gamma = 2$
- Using Erdös-Renyi method with $\lambda = 4$
- Using Erdös-Renyi method with $\lambda = 5$
- Using Erdös-Renyi method with $\lambda = 6$

In [36]:
input_index = 1

#create 20 sequences for each setting and write inputs
for num_vertex in [25,50,75,100]:
    
    sequences_scalefree = [seq_generator_scalefree(num_vertex, 2, i) for i in range(20)]
    for sequence in sequences_scalefree:
        write_input(input_index,'scale_free_2', sequence)
        input_index = input_index + 1
        
    sequences_erdos_renyi = [seq_generator_erdos_renyi(num_vertex, 4, i) for i in range(20)]
    for sequence in sequences_erdos_renyi:
        write_input(input_index,'erdos_renyi_4', sequence)
        input_index = input_index + 1
    
    
    sequences_erdos_renyi2 = [seq_generator_erdos_renyi(num_vertex, 5, i) for i in range(20)]
    for sequence in sequences_erdos_renyi2:
        write_input(input_index,'erdos_renyi_5', sequence)
        input_index = input_index + 1
    
    sequences_erdos_renyi3 = [seq_generator_erdos_renyi(num_vertex, 6, i) for i in range(20)]
    for sequence in sequences_erdos_renyi3:
        write_input(input_index,'erdos_renyi_6', sequence)
        input_index = input_index + 1

# Output Generation

We first start by reading the input files. To preserve the informations such as the input index and generation method, the output of this function is a tuple whose first element is the index of the input and the 2nd element is a dictionary containing the sequence, generation method, and parameter used to generate it

In [17]:
#Read an input sequence from txt and return its characteristics
def read_input(input_file):
    file_name = f'Inputs/{input_file}'
    
    #input name is of the form 'Group ID – Length –Input index- Method.txt' hence the 3rd element when we split it using '-'s. 
    sequence_index = file_name.split('-')[2]
    
    if 'scale_free' in file_name:
        sequence_generation_method = 'Scale-free'
        sequence_generation_parameter = 2
    else: 
        sequence_generation_method = 'Erdös-Renyi'
        for param in [4,5,6]:
            if f'erdos_renyi_{param}' in file_name:
                sequence_generation_parameter = param
                
    #sequence is at the second line
    with open(file_name, 'r') as f:
        sequence = f.read().split('\n')[1]
        
    f.close()
    
    sequence = np.array(sequence.split(), dtype = int)
    
    return (int(sequence_index), {'sequence': sequence,
           'number of vertices': len(sequence),
           'method': sequence_generation_method,
           'parameter': sequence_generation_parameter})

In [18]:
sequence_dicts = {}
#dictionary of sequences and their indeces
for file in [f for f in listdir('Inputs') if 'txt' in f]:
    index, sequence_dict = read_input(file)
    sequence_dicts[index] = sequence_dict

In [38]:
realization_dicts = {}

#Iterate over inputs 
for key in tqdm(range(1, 321)):
    #take the sequence
    sequence = sequence_dicts[key]['sequence']
    
    # is it graphical
    graphical = is_graphical(sequence)
    
    #initialize realization parameters
    HH_runtime = None
    PM_runtime = None
    SA_runtime = None
    hh_list = None
    pm_list = None
    sa_list = None

    #if it is graphical, update the realization parameters accordingly
    if is_graphical(sequence):

        t = time.time()
        hh_list = havel_hakimi_realization(sequence,[])
        HH_runtime = time.time() - t

        t = time.time()
        pm_list = pairing_model(sequence, t)
        PM_runtime = time.time() - t

        t = time.time()
        sa_list = seq_alg(sequence,[])
        SA_runtime = time.time() - t
    
    #Add the parameters to the dictionary with the corresponding index as the key
    realization_dicts[key] = {'graphical': is_graphical(sequence),
                          'Havel Hakimi Runtime': HH_runtime,
                          'Havel Hakimi Realization': hh_list,
                          'Pairing Model Runtime': PM_runtime,
                          'Pairing Model Realization': pm_list, 
                          'Sequential Algorithm Runtime': SA_runtime,
                          'Sequential Algorithm Realization':sa_list}  


100%|██████████| 320/320 [1:11:14<00:00, 13.36s/it]


In [40]:
#this code block is for dataframe generation
num_vertices = []
methods = []
parameters = []
graphicals = []
HH_runtimes = []
PM_runtimes = []
SA_runtimes = []

for key in tqdm(range(1, 321)):
    num_vertices.append(sequence_dicts[key]['number of vertices'])
    methods.append(sequence_dicts[key]['method'])
    parameters.append(sequence_dicts[key]['parameter'])
    graphicals.append(realization_dicts[key]['graphical'])
    HH_runtimes.append(realization_dicts[key]['Havel Hakimi Runtime'])
    PM_runtimes.append(realization_dicts[key]['Pairing Model Runtime'])
    SA_runtimes.append(realization_dicts[key]['Sequential Algorithm Runtime'])
    
output_runtime_df = pd.DataFrame(data = {'number of vertices': num_vertices,
                                        'method': methods,
                                        'parameter':parameters,
                                        'is graphical': graphicals,
                                        'Havel Hakimi Runtime': HH_runtimes,
                                        'Pairing Model Runtime': PM_runtimes,
                                        'Sequential Algorithm Runtime': SA_runtimes})

100%|██████████| 320/320 [00:00<00:00, 340740.61it/s]


In [41]:
output_runtime_df.head(20)

,number of vertices,method,parameter,is graphical,Havel Hakimi Runtime,Pairing Model Runtime,Sequential Algorithm Runtime
0,25,Scale-free,2,False,NaN,NaN,NaN
1,25,Scale-free,2,False,NaN,NaN,NaN
2,25,Scale-free,2,True,0.001020,0.022321,0.156170
3,25,Scale-free,2,False,NaN,NaN,NaN
4,25,Scale-free,2,False,NaN,NaN,NaN
5,25,Scale-free,2,False,NaN,NaN,NaN
6,25,Scale-free,2,True,0.000246,0.164526,0.176820
7,25,Scale-free,2,True,0.000189,0.038138,0.160929
8,25,Scale-free,2,False,NaN,NaN,NaN
9,25,Scale-free,2,False,NaN,NaN,NaN


In [42]:
output_runtime_df.groupby(['is graphical']).count()

,number of vertices,method,parameter,Havel Hakimi Runtime,Pairing Model Runtime,Sequential Algorithm Runtime
is graphical,,,,,,
False,181,181,181,0,0,0
True,139,139,139,139,139,139


In [45]:
output_runtime_df.drop(columns = 'is graphical').groupby(['method', 'number of vertices', 'parameter']).mean()

Havel Hakimi Runtime  \
method      number of vertices parameter                         
Erdös-Renyi 25                 4                      0.000671   
                               5                      0.000851   
                               6                      0.000973   
            50                 4                      0.001535   
                               5                      0.001843   
                               6                      0.001766   
            75                 4                      0.001722   
                               5                      0.001856   
                               6                      0.002039   
            100                4                      0.002522   
                               5                      0.002293   
                               6                      0.002726   
Scale-free  25                 2                      0.000464   
            50                 2                      0.001017   
            75                 2                      0.001403   
            100                2                      0.001407   

                                          Pairing Model Runtime  \
method      number of vertices parameter                          
Erdös-Renyi 25                 4                       0.070353   
                               5                       0.091597   
                               6                       0.145605   
            50                 4                       0.540575   
                               5                       0.495713   
                               6                       0.637126   
            75                 4                       1.155325   
                               5                       1.511439   
                               6                       2.204260   
            100                4                       2.475062   
                               5                       3.218559   
                               6                       4.420987   
Scale-free  25                 2                       0.068579   
            50                 2                       0.706646   
            75                 2                       0.754951   
            100                2                      84.720278   

                                          Sequential Algorithm Runtime  
method      number of vertices parameter                                
Erdös-Renyi 25                 4                              0.515941  
                               5                              0.736373  
                               6                              1.268097  
            50                 4                              7.437557  
                               5                              9.370237  
                               6                             12.018057  
            75                 4                             33.567782  
                               5                             44.633624  
                               6                             58.633327  
            100                4                             60.333882  
                               5                             60.352224  
                               6                             60.419963  
Scale-free  25                 2                              0.156107  
            50                 2                              2.271207  
            75                 2                              9.321313  
            100                2                             26.143967

# Writing Output

a for loop to write the outputs in the expected format. Note that Havel-Hakimi and sequential algorithms returns edge sets in our code, so we change them into adjacency lists before writing the outputs.

In [74]:
for key in tqdm(range(1, 321)):

    sequence = sequence_dicts[key]['sequence']
    graphical = realization_dicts[key]['graphical']

    for method in ['HH', 'PM','SA']:
        if graphical:

            if method == 'HH':
                edge_set = realization_dicts[key]['Havel Hakimi Realization']
                neigh_list = edge_set_to_neigborhood_list(sequence, edge_set)

            elif method == 'PM':
                neigh_list = realization_dicts[key]['Pairing Model Realization']

            elif method == 'SA':
                edge_set = realization_dicts[key]['Sequential Algorithm Realization']
                neigh_list = edge_set_to_neigborhood_list(sequence, edge_set)

            write_output(key, sequence_dicts[key]['sequence'], neigh_list, method)

        else:
            write_0(key, sequence_dicts[key]['sequence'], method)


100%|██████████| 320/320 [00:01<00:00, 176.48it/s]
